In [1]:
from selenium import webdriver
from time import sleep

        
class Scraper():
    def __init__(self, url):
        self.url = url
        self.driver = webdriver.Chrome(executable_path='chromedriver.exe')
        self.driver.get(self.url)
        sleep(3)
        #get number of pages
        try:
            page_ = self.driver.find_element_by_id('pagination')
            num_ = page_.find_element_by_id('inputPage').get_attribute('max-page')
            self.num_page = int(num_)
        except:
            self.num_page = 1
            
        
    #get news 
    def get_news(self):
        
        # title in class 'title' or 'title-detail' 
        try:
            self.title = self.driver.find_element_by_class_name('title').text
        except:
            pass
        
        try:
            self.title = self.driver.find_element_by_class_name('title-detail').text
        except:
            pass
        
        #description in class 'description'
        self.description = self.driver.find_element_by_class_name('description').text
        
        # class 'Normal' contain contents
        content_list =  self.driver.find_elements_by_class_name('Normal')
        
        # author's name in class 'author_mail' or last  'Nomal' 
        author_mail=None
        
        try:
            author_mail = self.driver.find_element_by_class_name('author_mail')
        except:
            pass
            
        self.content=''

        if author_mail == None:# last Normal maybe contain author's name
            
            #if the last 'Normal' can't find attribute 'style', it's not author's name
            # because author's name is bold
            
            check_author = content_list[-1].get_attribute('style')
            
            if check_author=='':
                self.author = 'None'
            else:
                self.author = content_list[-1].text
                
            
            # get contents and convert to text 
            for cnt in content_list[:len(content_list)-1]:
                cnt_text = cnt.text
                
                if cnt_text.find('>>')==-1:  # >>.... is a link so don't get this content
                    self.content += ' ' + cnt_text + '\n'
                else:
                    pass
    
        else:
            self.author = author_mail.text
            for cnt in content_list:
                cnt_text = cnt.text
                if cnt_text.find('>>')==-1:
                    self.content+= ' ' + cnt_text + '\n'
                else:
                    pass
        
    '''
    def print_news(self):
        print(self.title)
        print(self.description)
        print(self.content)
        print(self.author)
    '''    
    
    
    #get comments
    
    #click button 'view more'
    def click_view_more(self):
        try:
            show_cmt = self.driver.find_element_by_class_name('txt_666')
            show_cmt.click()
            sleep(5)
        except:
            pass
    
    #scroll up to top of comment box
    def scroll_up_to_top(self):
        begin = self.driver.find_element_by_class_name('active')
        self.driver.execute_script('arguments[0].scrollIntoView(true)',begin)
        sleep(5)
    
    #click all buttons 'view all reply' 
    def view_all_reply(self):
        try:
            show_rep = self.driver.find_elements_by_class_name('view_all_reply')
            for s in show_rep:
                s.click()
                sleep(2)
                try:
                    s.click()# click again to show full rep_cmt
                    sleep(2)
                except:
                    pass
        except:
            pass
        sleep(3)    
            
    def view_all_content_cmt(self, comment):# there are several comment with content less 
        # click icon 'show full comment' 
        try:
            show_more = comment.find_element_by_class_name('icon_show_full_comment')
            show_more.click()
            sleep(1)
        except:
            pass
        
        
    def process_page(self):
        # init comment 
        comments = []
        
        #click button viewmore
        self.click_view_more()
    
        #scroll up to top of comment box
        self.scroll_up_to_top()

        #click all buttons 'view all reply'
        self.view_all_reply()

        #scroll again
        self.scroll_up_to_top()

       
        #get infor from all comment

        cmt = self.driver.find_elements_by_class_name('content-comment')

        for c in cmt:
            
            #get poster's name
            poster = c.find_element_by_class_name('nickname').text
            
            #show full comment 
            self.view_all_content_cmt(c)

            #get all infor from comment 
            c_ = c.text # poster, content, block_like,...
            if c_=='':
                pass
            else:
                c_ = c_.split('\n')
                
                #there are several comment can't split poster and content by '\n'
                if c_[0]!=poster:
                    cnt_cmt = c_[0].replace(poster,'')
                else:
                    cnt_cmt = c_[1]
                    
            
                comments.append(poster + ' : ' + cnt_cmt)
            
        sleep(3)
        return comments
    def next_page(self):
        try:
            # find button nextpage : '>'
            page = self.driver.find_element_by_id('pagination')
            next_ = page.find_elements_by_tag_name('a')
            next_page = next_[1]
            
            #scroll to this area
            self.driver.execute_script('arguments[0].scrollIntoView(true)',next_page) 
            self.driver.execute_script("window.scrollTo(0, window.scrollY - 200)")
            sleep(2)
            
            #click 
            next_page.click()
            sleep(3)
        
        except:
            pass
    
    def get_all_comment(self):
        
        #init all_comments 
        self.all_comments = []
        
        
        #get comment from fist page
        self.all_comments += self.process_page()
        #if num_page>1:
        for i in range(self.num_page-1):
            #move to next page
            self.next_page()
            #get comment from next page
            self.all_comments += self.process_page()
            sleep(3)
    '''        
    def print_cmt(self):
        for cmt in self.all_comments:
            print(cmt)
    '''        
    
    def write_file(self, getcmt = True):
        
        with open('output','w',encoding = 'utf-8') as f:
            f.write('-----NEWS-----\n\n')
            f.write('title: ' +self.title+ '\n')
            f.write('description: ' + self.description +'\n')
            f.write('content: ' + self.content +'\n')
            f.write('author: ' + self.author + '\n')
            if getcmt:
                f.write('\n-----Comment-----\n\n')
                for cmt in self.all_comments:
                    f.write(cmt + '\n')
        print("done")
        
        
        
        
        
        
        
        
        
        
x=Scraper('https://vnexpress.net/bo-tai-nguyen-amp-moi-truong-de-nghi-so-do-ghi-ten-ca-vo-chong-4305678.html')

x.get_news()
x.get_all_comment()
x.write_file()



done
